# Zajęcia 1 (część 2)

## Wczytanie danych

Bedziemy wykorzystywać danye z ankiety StackOverflow z 2020.

https://insights.stackoverflow.com/survey/

Dane sa dostepne na google drive. Skorzystamy z modułu GoogleDriveDownloader, ktory pozwala pobrac dokument o podanym id.


In [18]:
! pip install googledrivedownloader

In [19]:
from google_drive_downloader import GoogleDriveDownloader as gdd

In [20]:
from pathlib import Path
path_dir = str(Path.home()) + "/data/2020/"  # ustawmy sciezke na HOME/data/2020
archive_dir = path_dir + "survey.zip"        # plik zapiszemy pod nazwa survey.zip

In [21]:
path_dir

'/root/data/2020/'

In [22]:
archive_dir

'/root/data/2020/survey.zip'

In [23]:
# sciagniecie pliku we wskazane miejsce
gdd.download_file_from_google_drive(file_id='1dfGerWeWkcyQ9GX9x20rdSGj7WtEpzBB',
                                    dest_path=archive_dir,
                                    unzip=True)

## <span style='background:yellow'> ZADANIE 1 </span>
Zapoznaj sie z plikami tekstowymi (survey_results_public.csv oraz survey_results_schema.csv). Podejrzyj ich zawartość, sprawdź ich wielkość (liczba linii oraz rozmiar). Wgraj plik do swojego kubełka `gs://ds-$SEMESTER-$USER_ID-notebook-data` na GCS do katalogu `survey/2020/`. Jesli nie masz kubełka stwórz go.


In [24]:
# TODO Please update user id
%env USER_ID=7008
%env SEMESTER=2024l

env: USER_ID=7008
env: SEMESTER=2024l


In [25]:
! gsutil cp /root/data/2020/survey_results_public.csv gs://ds-$SEMESTER-$USER_ID-notebook-data/survey/2020/

Copying file:///root/data/2020/survey_results_public.csv [Content-Type=text/csv]...
| [1 files][ 90.2 MiB/ 90.2 MiB]                                                
Operation completed over 1 objects/90.2 MiB.                                     


##  Podłączenie do sesji Spark na GKE


#### WAZNE
jesli w poprzednim notatniku masz aktywną sesję Spark zakończ ją (w poprzednim notatniku) poleceniem spark.stop()

In [32]:
from pyspark.sql import SparkSession
#spark.stop()
spark = SparkSession \
.builder \
.config("spark.executor.instances", "1")\
.config('spark.driver.memory','1g')\
.config('spark.executor.memory', '1g') \
.getOrCreate()

In [ ]:
spark.stop()

## Dostęp do danych na GCS

In [27]:
# ścieżka dostępu do pliku na GCS
# TODO Please update user id
user_id = 9903
semester = '2024l'
gs_path = f'gs://ds-{semester}-{user_id}-notebook-data/survey/2020/survey_results_public.csv'

In [28]:
gs_path

'gs://ds-2024l-9903-notebook-data/survey/2020/survey_results_public.csv'

## Spark SQL

Platforma Apache Spark posiada komponent Spark SQL, który pozwala traktować dane jak tabele w bazie danych. Można zakładać swoje schematy baz danych oraz korzystać z języka SQL.

In [29]:
table_name = "survey_2020"                               # nazwa tabeli ktora bedziemy chcieli stworzyc

In [33]:
spark.sql(f'DROP TABLE IF EXISTS {table_name}')       # usun te tabele jesli istniala wczesniej 

# stworz tabele korzystajac z danych we wskazanej lokalizacji
spark.sql(f'CREATE TABLE IF NOT EXISTS {table_name} \
          USING csv \
          OPTIONS (HEADER true, INFERSCHEMA true) \
          LOCATION "{gs_path}"')

25/01/06 16:42:02 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: gs://ds-2024l-9903-notebook-data/survey/2020/survey_results_public.csv.
java.io.IOException: Error accessing gs://ds-2024l-9903-notebook-data/survey/2020/survey_results_public.csv
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getObject(GoogleCloudStorageImpl.java:2181) ~[gcs-connector-3.0.3.jar:?]
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getItemInfo(GoogleCloudStorageImpl.java:2070) ~[gcs-connector-3.0.3.jar:?]
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystemImpl.getFileInfoInternal(GoogleCloudStorageFileSystemImpl.java:1030) ~[gcs-connector-3.0.3.jar:?]
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystemImpl.getFileInfoInternal(GoogleCloudStorageFileSystemIm

Py4JJavaError: An error occurred while calling o69.sql.
: java.io.IOException: Error accessing gs://ds-2024l-9903-notebook-data/survey/2020/survey_results_public.csv
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getObject(GoogleCloudStorageImpl.java:2181)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getItemInfo(GoogleCloudStorageImpl.java:2070)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystemImpl.listFileInfo(GoogleCloudStorageFileSystemImpl.java:931)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem.lambda$listStatus$12(GoogleHadoopFileSystem.java:828)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.invokeTrackingDuration(IOStatisticsBinding.java:547)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.lambda$trackDurationOfOperation$5(IOStatisticsBinding.java:528)
	at org.apache.hadoop.fs.statistics.imp

## Weryfikacja danych 
Sprawdzmy zaczytane dane.

In [31]:
spark.sql(f"describe {table_name}").show() # nie wszystkie dane ...

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `survey_2020` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 9;
'DescribeRelation false, [col_name#0, data_type#1, comment#2]
+- 'UnresolvedTableOrView [survey_2020], DESCRIBE TABLE, true


In [ ]:
spark.sql(f"describe {table_name}").show(100, truncate=False) # niepoprawne typy danych... "NA" 

In [ ]:
spark.sql(f"SELECT DISTINCT Age FROM {table_name} ORDER BY Age DESC").show()

## Obsługa wartosci 'NA' - ponowne stworzenie tabeli

In [ ]:
spark.sql(f'DROP TABLE IF EXISTS {table_name}')

# wykorzystujemy dodatkową opcję: NULLVALUE "NA"
spark.sql(f'CREATE TABLE IF NOT EXISTS {table_name} \
          USING csv \
          OPTIONS (HEADER true, INFERSCHEMA true, NULLVALUE "NA") \
          LOCATION "{gs_path}"')


In [ ]:
spark.sql(f"DESCRIBE {table_name}").show(100)

In [ ]:
spark.sql(f"SELECT DISTINCT  Age FROM {table_name} ORDER BY Age DESC").show()

In [ ]:
# sprawdzenie liczności tabeli
spark.sql(f"select count(*) from {table_name}").show()

In [ ]:
spark.sql(f"select count(*) from {table_name}").explain()  # tak jak na poprzednich zajeciach mozemy wygenerowac plany wykonania polecenia

## Podgląd danych

In [ ]:
spark.sql(f"select * from {table_name}").show()

## Biblioteka Pandas

https://pandas.pydata.org/

Moduł Pandas jest biblioteką Pythonową do manipulacji danymi. W szczegolnosci w pandas mozemy stworzyc ramki danych i wykonywac na niej analize, agregacje oraz wizualizacje danych. 
Przy nieduzych zbiorach danych i prostych operacjach to doskonała biblioteka. Jednak kiedy zbior danych sie rozrasta lub kiedy wymagane sa zlozone transformacje to operacje moga byc wolne.

Operacje na rozproszonych danych sa szybsze. Ale tu takze napotykamy ograniczenia np trudność w wizualizacji danych.

In [ ]:
import pandas as pd

In [ ]:
spark.sql(f"select * from {table_name} limit 10").toPandas()

**Ważne** 

Metoda toPandas() na ramce pyspark, konwertuje ramkę pyspark do ramki pandas. Wykonuje akcje pobrania (collect) wszystkich danych z executorów (z JVM) i transfer do  programu sterujacego (driver) i konwersje do typu Pythonowego w notatniku. Ze względu na ograniczenia pamięciowe w programie sterującym należy to wykonywać na podzbiorach danych.

**DataFrame.collect() collects the distributed data to the driver side as the local data in Python. Note that this can throw an out-of-memory error when the dataset is too large to fit in the driver side because it collects all the data from executors to the driver side.**

**Note that DataFrame.toPandas() results in the collection of all records in the DataFrame to the driver program and should be done on a small subset of the data.**

In [ ]:
dist_df = spark.sql(f"select * from {table_name} LIMIT 10")
local_df = spark.sql(f"select * from {table_name} LIMIT 10").toPandas()

In [ ]:
type(dist_df)  # dataframe Sparkowy ("przepis na dane, rozproszony, leniwy")

In [ ]:
type(local_df)  # dataframe Pandasowy (lokalny, sciągnięty do pamięci operacyjnej)

In [ ]:
dist_df.show()

In [ ]:
local_df

In [ ]:
pd.set_option('display.max_columns', None)    # pokazuj wszystkie kolumny
# pd.reset_option(“display.max_columns”)

## <span style='background:yellow'> ZADANIE 2 </span>
Napisz w Spark SQL zapytanie które zwróci średnią liczbę godzin przepracowywanych przez z respondentów pogrupowanych ze względu na kraj. Następnie przekształć wynik do ramki pandasowej i ją wyświetl.


## Wizualizacje

Do wizualizacji będziemy się posługiwać modułami matplotlib (https://matplotlib.org/) i seaborn (https://seaborn.pydata.org/). Do bardzo rozbudowane moduły, zachęcamy do eksploracji oficjalnych dokumentacji. Na zajęciach zrealizujemy następujące wykresy:
* histogramy
* liniowe 
* wiolinowe
* kołowe 

Moduły wizualizacyjne wymagają danych na lokalnej maszynie. Mogą być to natywne typy danych Pythonowe (słowniki, listy) ale także np ramki danych pandasowe. ~~Nie działa wizualizacja na ramkach danych Sparkowych.~~


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## Narysuj histogram wieku respondentów

In [ ]:
# przygotowanie danych
# przycinamy dane tylko do zakresu ktory jest potrzebny do realizacji polecenia
ages = spark.sql(f"SELECT CAST (Age AS INT) \
                    FROM {table_name} \
                    WHERE age IS NOT NULL \
                    AND age BETWEEN 10 AND 80").toPandas()

In [ ]:
ages

In [ ]:
ages.hist("Age", bins=10)
plt.show()

In [ ]:
sns.displot(ages, bins=10, rug=True, kde=False)
plt.show()

## Jaki jest udział programistów hobbistów? Narysuj wykres kołowy

Będzie nas interesowała ta proporcja ze względu na płeć.

In [ ]:
# przygotowanie (filtrowanie, grupowanie i zliczenie) danych na rozproszonych danych (spark sql)
# pozniej pobranie do pandasowej ramki
hobby_all = spark.sql(f"SELECT Hobbyist, COUNT(*) AS cnt FROM {table_name} WHERE Hobbyist IS NOT NULL GROUP BY Hobbyist").toPandas()
hobby_men = spark.sql(f"SELECT Hobbyist, COUNT(*) AS cnt FROM {table_name} WHERE Hobbyist IS NOT NULL AND Gender='Man' GROUP BY Hobbyist").toPandas()
hobby_women = spark.sql(f"SELECT Hobbyist, COUNT(*) AS cnt FROM {table_name} WHERE Hobbyist IS NOT NULL AND Gender='Woman' GROUP BY Hobbyist").toPandas()


In [ ]:
hobby_all.plot.pie(y='cnt', labels=hobby_all['Hobbyist'], title="All", autopct='%.0f')
plt.legend(loc="lower center")
plt.show()

In [ ]:
hobby_men.plot.pie(y='cnt', labels=hobby_men['Hobbyist'], title="Men", autopct='%.0f')
plt.legend(loc="lower center")
plt.show()

In [ ]:
hobby_women.plot.pie(y='cnt', labels=hobby_women['Hobbyist'], title="Women", autopct='%.0f')
plt.legend(loc="lower center")
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

hobby_all.plot.pie(y='cnt', labels=hobby_all['Hobbyist'], title="All", ax=axes[0], autopct='%.0f')
hobby_men.plot.pie(y='cnt', labels=hobby_men['Hobbyist'], title="Men", ax=axes[1], autopct='%.0f')
hobby_women.plot.pie(y='cnt', labels=hobby_men['Hobbyist'], title="Women", ax=axes[2], autopct='%.0f')

plt.show()

## Wykres liniowy. Zależność między wiekiem a liczbą przepracowanych godzin
Interesują nas dla developerzy profesjonaliści (nie hobbiści) w przedziale wiekowym 18-65.

In [ ]:
# przygotowanie (filtrowanie, grupowanie, wyliczenie sredniej oraz sortowanie) danych na rozproszonych danych (spark sql)
# pozniej pobranie do pandasowej ramki

age_work = spark.sql(f"SELECT age, CAST (avg(WorkWeekHrs) AS INT) AS avg FROM {table_name} \
            WHERE WorkWeekHrs IS NOT NULL AND age BETWEEN 18 AND 65 AND hobbyist = 'No' \
            GROUP BY age \
            ORDER BY age ASC").toPandas()

In [ ]:
age_work

In [ ]:
age_work.plot(x='age', y='avg', kind='line')
plt.show()

In [ ]:
sns.relplot(x="age", y="avg", kind="line", data=age_work);
plt.show()

## Wykres słupkowy. Pokaż liczbę respondentów na kraj

Interesuje nas tylko 10 krajów o najwyższej liczbie respondentow.

In [ ]:
# przygotowanie (grupowanie, zliczenie, sortowanie oraz przyciecie do 10 wyników) danych na rozproszonych danych (spark sql)
# pozniej pobranie do pandasowej ramki

max_countries = spark.sql(f"SELECT country, COUNT(*) AS cnt \
                FROM {table_name} \
                GROUP BY country \
                ORDER BY cnt DESC \
                LIMIT 10 ").toPandas()

In [ ]:
max_countries.plot.bar(y='cnt', x='country')
plt.show()

In [ ]:
sns.catplot(x="country", y="cnt", kind="bar",\
            data=max_countries).set_xticklabels(rotation=65)
plt.show()

## Wykres słupkowy. Średnie zarobki w  krajach w ktorych jest powyzej 1000 respondentów

In [ ]:
# przygotowanie (filtrowanie, grupowanie, wyliczenie sredniej, filtrowanie po liczności i grup oraz sortowanie) danych na rozproszonych danych (spark sql)
# pozniej pobranie do pandasowej ramki

country_salary = spark.sql(f"SELECT country, \
    CAST (avg(ConvertedComp) AS INT) as avg \
    FROM {table_name} \
    WHERE country IS NOT NULL \
    GROUP BY country \
    HAVING COUNT(*) > 1000 \
    ORDER BY avg DESC ").toPandas()

In [ ]:
country_salary.plot.barh(("country"))
plt.show()

## Boxplot. Pokaz rozklad pensji w krajach gdzie jest powyzej 1000 respondentów
Tutaj będziemy musieli skorzystać z podzapytania.

In [ ]:
# przygotowanie danych na rozproszonych danych (spark sql). Mamy tu do czynienia z podzapytaniem
# pozniej pobranie do pandasowej ramki

country_comp = spark.sql(f"SELECT country, CAST(ConvertedComp AS INT) \
                FROM {table_name} \
                WHERE country IN (SELECT country FROM {table_name} GROUP BY country HAVING COUNT(*) > 1000) \
                AND ConvertedComp IS NOT NULL AND ConvertedComp > 0 \
                ORDER BY ConvertedComp desc").toPandas()
country_comp

In [ ]:
country_comp.boxplot(column="ConvertedComp", by="country", \
                     showfliers=False, rot=60, meanline=True)
plt.show()

In [ ]:
sns.catplot(x="country", y="ConvertedComp", kind="box", \
            showfliers=False, data=country_comp, palette="Blues")\
    .set_xticklabels(rotation=65)
plt.show()

## <span style='background:yellow'> ZADANIE 3 </span>
Narysuj rozklad pensji w zaleznosci od plci.

## Narysuj wykres popularnosci jezykow programowania

In [ ]:
spark.sql(f"select LanguageWorkedWith from {table_name} where LanguageWorkedWith IS NOT NULL").show(truncate=False)

Języki programowania są zapisane w pojedynczej komórce. Będzie trzeba je rozdzielić i zliczyć. Tak przygotowane dane dopiero posłużą nam do narysowania wykresu. Wykorzystamy funkcję `posexplode`.

In [ ]:
langs = spark.sql(f"select LanguageWorkedWith from {table_name} where LanguageWorkedWith IS NOT NULL")

In [ ]:
from pyspark.sql.functions import *

langs.select(
        posexplode(split("LanguageWorkedWith", ";")).alias("pos", "language")
    ).show()

In [ ]:
langs.select(
        posexplode(split("LanguageWorkedWith", ";")).alias("pos", "language")).groupBy("language").count().orderBy("count").show()

In [ ]:
langs_pd = langs.select(
        posexplode(split("LanguageWorkedWith", ";")).alias("pos", "language")).groupBy("language").count().orderBy("count").toPandas()
langs_pd

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(8, 9))
plt.barh(width=langs_pd["count"], y=langs_pd["language"])
plt.show()

## Narysuj wykres popularnosci jezykow wsrod Data Scientists


Zdefiniujmy sobie funkcję, która przekształca nam języki do wymaganej przez nas postaci

In [ ]:
from pyspark.sql.functions import *

def prepare_lang(df, colName='LanguageWorkedWith'):
    summary = df.select(posexplode(split(colName, ";")).alias("pos", "language")).groupBy("language").count().orderBy("count")
    return summary

In [ ]:
langs_ds = spark.sql(f"SELECT LanguageWorkedWith \
                FROM {table_name} \
                WHERE DevType LIKE '%Data scientist%'")

sum_lang = prepare_lang(langs_ds).toPandas()

figure(figsize=(8, 9))
plt.barh(width=sum_lang["count"], y=sum_lang["language"])
plt.show()

## Narysuj wykres którego chcą wykorzystywać w przyszłości Data Scientists

In [ ]:
lang_desired = spark.sql(f"select LanguageDesireNextYear \
                from {table_name} \
                where DevType like '%Data scientist%'")

sum_lang = prepare_lang(lang_desired, 'LanguageDesireNextYear').toPandas()

figure(num=None, figsize=(8, 9), dpi=80, facecolor='w', edgecolor='k')
plt.barh(width=sum_lang["count"], y=sum_lang["language"])
plt.show()

## Narysuj wykres prezentujący liczbę godzin na pracy w zależności od wykształcenia

In [ ]:
spark.sql(f"select distinct EdLevel from {table_name}").show(truncate=False) # jakie są wartości wykształcenia

In [ ]:

ed_pandas = spark.sql(f"SELECT EdLevel, WorkWeekHrs FROM {table_name} \
            WHERE WorkWeekHrs BETWEEN 10 AND 80 \
            AND (EdLevel LIKE '%Bachelor%' OR EdLevel LIKE '%Master%' OR EdLevel LIKE '%Other doctoral%')").toPandas()

ed_pandas['EdLevel'] = ed_pandas['EdLevel'].replace('Bachelor’s degree (B.A., B.S., B.Eng., etc.)','Bachelor')
ed_pandas['EdLevel'] = ed_pandas['EdLevel'].replace('Master’s degree (M.A., M.S., M.Eng., MBA, etc.)','Master')
ed_pandas['EdLevel'] = ed_pandas['EdLevel'].replace('Other doctoral degree (Ph.D., Ed.D., etc.)','Doctor')

ed_pandas

In [ ]:
sns.catplot(x="EdLevel", y="WorkWeekHrs", data=ed_pandas)
plt.show()

## Narysuj wykres wiolinowy pokazujacy rozkład dochodów w zależności od wykształcenia

In [ ]:
ed_pay = spark.sql(f"SELECT EdLevel, CAST (CompTotal AS INT) AS CompTotal FROM {table_name} \
            WHERE CompTotal BETWEEN 0 AND 1000000  \
            AND (EdLevel LIKE '%Bachelor%' OR EdLevel LIKE '%Master%' OR EdLevel LIKE '%Other doctoral%')").toPandas()

ed_pay['EdLevel'] = ed_pay['EdLevel'].replace('Bachelor’s degree (B.A., B.S., B.Eng., etc.)','Bachelor')
ed_pay['EdLevel'] = ed_pay['EdLevel'].replace('Master’s degree (M.A., M.S., M.Eng., MBA, etc.)','Master')
ed_pay['EdLevel'] = ed_pay['EdLevel'].replace('Other doctoral degree (Ph.D., Ed.D., etc.)','Doctor')

ed_pay

In [ ]:
sns.catplot(x="EdLevel", y="CompTotal", kind="boxen",
            data=ed_pay);
plt.show()

## ⭐ Narysuj heatmape odwiedzin na StackOverflow dla wybranych krajów ⭐

In [ ]:
spark.sql(f"SELECT DISTINCT SOVisitFreq FROM {table_name}").show(truncate=False)

In [ ]:
so_v = spark.sql(f"SELECT SOVisitFreq, t1.country, COUNT(*)/first(t2.t) AS cnt from {table_name} t1 \
            JOIN (SELECT country, COUNT(*) as t FROM {table_name} GROUP BY country) t2 \
            ON t1.country = t2.country \
            WHERE t1.country IS NOT NULL AND SOVisitFreq IS NOT NULL \
            AND t1.country IN ('Poland', 'United States', 'Russian Federation', 'China', 'India', 'Germany', 'Japan') \
            GROUP BY t1.country, SOVisitFreq").toPandas()

so_v['SOVisitFreq'] = pd.Categorical(so_v['SOVisitFreq'], ["I have never visited Stack Overflow (before today)", "Less than once per month or monthly", "A few times per month or weekly", "A few times per week", "Daily or almost daily", "Multiple times per day"])
# so_v.sort_values['SOVisitFreq']


In [ ]:
heatmap2_data = pd.pivot_table(so_v, values='cnt', index=['country'], columns='SOVisitFreq')
sns.heatmap(heatmap2_data, cmap="BuGn")
plt.show()

## <span style='background:yellow'> ZADANIE 4 </span>
* Narysuj wykres słupkowy popularności wykorzystywanych baz danych przez profesjonalnych programistów.
* Narysuj wykres kołowy przedstawiający procentowy udział poziomu wykształcenia inz, mgr i dr w grupie respondentów.

In [36]:
spark.stop()